In [1]:
import pandas as pd
import time

In [2]:
path_train = "../resource/PINGAN-2018-train_demo.csv"
path_test = "../resource/PINGAN-2018-test_demo.csv"

In [3]:
def load_data(path_train,path_test):
    train_data = pd.read_csv(path_train)
    test_data = pd.read_csv(path_test)
    return train_data,test_data

In [4]:
#加载数据
train_data,test_data=load_data(path_train,path_test)

In [5]:
#拼接训练集和测试集进行特征工程
test_data['TERMINALNO'] = test_data['TERMINALNO']+train_data['TERMINALNO'].max()
data = pd.concat([train_data,test_data])

In [6]:
#重置index
data.reset_index(drop=True,inplace=True)

In [7]:
#时间处理
def time_datetime(value):
    format = '%Y%m%d%H%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_date(value):
    format = '%Y%m%d'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_hour(value):
    format = '%H'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)
def time_minute(value):
    format = '%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

#转换成时刻
data['time']=data['TIME'].apply(time_datetime)
data['date']=data['TIME'].apply(time_date)
data['hour']=data['TIME'].apply(time_hour)
data['minute']=data['TIME'].apply(time_minute)

In [8]:
# trip_max
feature=pd.DataFrame()
feature[['TERMINALNO','trip_max']]=pd.DataFrame(data['TRIP_ID'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','TRIP_ID']]

In [9]:
# lon_max lon_min lon
lonmax=pd.DataFrame()
lonmin=pd.DataFrame()
lonmax[['TERMINALNO','lon_max']]=pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','LONGITUDE']]
lonmin[['TERMINALNO','lon_min']]=pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','LONGITUDE']]
feature=pd.merge(feature,lonmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,lonmin,how='left',on='TERMINALNO')
feature['lon']=feature['lon_max']-feature['lon_min']

In [10]:
# lat_max lat_min lat
latmax=pd.DataFrame()
latmin=pd.DataFrame()
latmax[['TERMINALNO','lat_max']]=pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','LATITUDE']]
latmin[['TERMINALNO','lat_min']]=pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','LATITUDE']]
feature=pd.merge(feature,latmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,latmin,how='left',on='TERMINALNO')
feature['lat']=feature['lat_max']-feature['lat_min']

In [11]:
# heg_max heg_min heg_mean heg
hegmax=pd.DataFrame()
hegmin=pd.DataFrame()
hegmean=pd.DataFrame()
hegmax[['TERMINALNO','heg_max']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','HEIGHT']]
hegmin[['TERMINALNO','heg_min']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','HEIGHT']]
hegmean[['TERMINALNO','heg_mean']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).mean()).reset_index()[['TERMINALNO','HEIGHT']]
feature=pd.merge(feature,hegmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,hegmin,how='left',on='TERMINALNO')
feature=pd.merge(feature,hegmean,how='left',on='TERMINALNO')
feature['heg']=feature['heg_max']-feature['heg_min']

In [12]:
# volu 活动区间体积
feature['vol']=feature['lon']*feature['lat']*feature['heg']

In [13]:
# 速度 sp_max sp_mean
spmax=pd.DataFrame()
spmean=pd.DataFrame()
spmax[['TERMINALNO','sp_max']]=pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','SPEED']]
spmean[['TERMINALNO','sp_mean']]=pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).mean()).reset_index()[['TERMINALNO','SPEED']]
feature=pd.merge(feature,spmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,spmean,how='left',on='TERMINALNO')

In [14]:
data.head()

,CALLSTATE,DIRECTION,HEIGHT,LATITUDE,LONGITUDE,SPEED,TERMINALNO,TIME,TRIP_ID,Y,time,date,hour,minute
0,0,12,39.402588,41.103741,122.985168,2.15,1,1476923580,1,0.0,201610200833,20161020,8,33
1,0,24,39.311157,41.104904,122.984398,4.11,1,1476923640,1,0.0,201610200834,20161020,8,34
2,0,74,34.178955,41.106388,122.986496,2.99,1,1476923700,1,0.0,201610200835,20161020,8,35
3,0,115,37.765381,41.106884,122.989769,7.59,1,1476923760,1,0.0,201610200836,20161020,8,36
4,0,151,36.049194,41.105442,122.991089,0.24,1,1476923820,1,0.0,201610200837,20161020,8,37


In [15]:
#callstate
call0=pd.DataFrame()
call1=pd.DataFrame()
call0[['TERMINALNO','call0']]=pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] == 0].groupby(data['TERMINALNO']).count()).reset_index()[['TERMINALNO','CALLSTATE']]
call1[['TERMINALNO','call1']]=pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] > 0].groupby(data['TERMINALNO']).count()).reset_index()[['TERMINALNO','CALLSTATE']]
feature=pd.merge(feature,call0,how='left',on='TERMINALNO')
feature=pd.merge(feature,call1,how='left',on='TERMINALNO')

In [16]:
feature['call0'].fillna(0,inplace=True)
feature['call1'].fillna(0,inplace=True)
feature['call_ratio_0']=feature['call0']/(feature['call0']+feature['call1'])
feature['call_ratio_1']=feature['call1']/(feature['call0']+feature['call1'])

In [17]:
# feature['call_ratio_0'].fillna(feature['call_ratio_0'].mean(),inplace=True)
feature

,TERMINALNO,trip_max,lon_max,lon_min,lon,lat_max,lat_min,lat,heg_max,heg_min,heg_mean,heg,vol,sp_max,sp_mean,call0,call1,call_ratio_0,call_ratio_1
0,1,45,124.809792,122.971771,1.838021,43.506298,41.082325,2.423973,224.069580,-7.248444,47.848093,231.318024,1030.594264,32.779999,17.489840,144.0,150.0,0.489796,0.510204
1,2,66,117.079720,113.330002,3.749718,30.790524,22.917767,7.872757,526.300537,-112.462982,71.400718,638.763519,18856.694241,36.119999,9.287734,289.0,439.0,0.396978,0.603022
2,3,37,113.918510,112.899796,1.018714,28.856243,22.474741,6.381502,125.748291,-10.797699,43.942672,136.545990,887.675299,25.440001,7.987331,838.0,125.0,0.870197,0.129803
3,4,98,116.943901,116.751045,0.192856,33.992123,33.473557,0.518566,115.885498,-14.475311,31.659250,130.360809,13.037197,33.310001,6.312753,181.0,611.0,0.228535,0.771465
4,5,85,115.024368,114.194641,0.829727,31.188187,30.531494,0.656693,117.702576,-14.855835,29.461915,132.558411,72.227885,53.480000,7.695846,903.0,0.0,1.000000,0.000000
5,6,40,120.920166,120.510963,0.409203,31.661022,31.277920,0.383102,130.427490,-18.174103,9.389893,148.601593,23.295750,29.570000,11.456653,478.0,0.0,1.000000,0.000000
6,7,138,108.436508,106.474197,1.962311,30.864641,29.538458,1.326183,432.401733,162.777222,246.410570,269.624511,701.666376,35.680000,5.705086,321.0,1132.0,0.220922,0.779078
7,8,59,122.006638,121.575500,0.431138,39.647110,38.863209,0.783901,188.175537,-13.241180,36.571975,201.416717,68.072709,42.720001,9.100117,662.0,36.0,0.948424,0.051576
8,9,9,121.155373,120.154106,1.001267,31.774067,30.354998,1.419069,32.210754,-17.095184,5.575440,49.305938,70.057178,35.680000,16.274006,317.0,0.0,1.000000,0.000000
9,10,57,106.610077,105.911392,0.698685,29.581587,29.359980,0.221607,400.146973,175.481934,305.556835,224.665039,34.785671,32.330002,7.297735,3.0,641.0,0.004658,0.995342


In [ ]:
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):  # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    r = 6371  # 地球平均半径，单位为公里
    return c * r

def cal_dis(value):
    value = value.reset_index(drop=True)
    dis = 0
    for index, row in value.iterrows():
        if index != 0:
            temp_dis = haversine(value['LONGITUDE'].iloc[index-1], value['LATITUDE'].iloc[index-1],
                                 row['LONGITUDE'], row['LATITUDE'])
            dis += temp_dis
    return dis

def cal_distance(value):
    dis = pd.DataFrame()
    value.sort_values(by='date')
    dis[['dis']] = pd.DataFrame(value[['LONGITUDE', 'LATITUDE']].groupby(value['date']).apply(cal_dis))
    distance = dis['dis'].sum()
    del dis
    return distance

In [18]:
train_data

,TERMINALNO,TIME,TRIP_ID,LONGITUDE,LATITUDE,DIRECTION,HEIGHT,SPEED,CALLSTATE,Y
0,1,1476923580,1,122.985168,41.103741,12,39.402588,2.15,0,0.0
1,1,1476923640,1,122.984398,41.104904,24,39.311157,4.11,0,0.0
2,1,1476923700,1,122.986496,41.106388,74,34.178955,2.99,0,0.0
3,1,1476923760,1,122.989769,41.106884,115,37.765381,7.59,0,0.0
4,1,1476923820,1,122.991089,41.105442,151,36.049194,0.24,0,0.0
5,1,1476939900,2,122.977905,41.103439,17,60.467529,1.42,4,0.0
6,1,1476939960,3,122.979752,41.107075,17,22.702515,3.96,4,0.0
7,1,1476940020,3,122.981956,41.109329,123,35.781616,4.66,4,0.0
8,1,1476940080,3,122.982986,41.108887,99,61.232910,0.64,4,0.0
9,1,1476940140,3,122.983658,41.108940,16,82.899170,4.68,4,0.0


In [19]:
#训练集和测试集去重
train_data.drop_duplicates(subset='TERMINALNO',inplace=True)
test_data.drop_duplicates(subset='TERMINALNO',inplace=True)

In [20]:
train_data

,TERMINALNO,TIME,TRIP_ID,LONGITUDE,LATITUDE,DIRECTION,HEIGHT,SPEED,CALLSTATE,Y
0,1,1476923580,1,122.985168,41.103741,12,39.402588,2.150000,0,0.000000
294,2,1481711280,1,116.157593,24.658466,174,113.864014,7.770000,4,0.000000
1022,3,1471742280,1,112.922943,28.820982,315,30.797756,10.760000,0,0.000000
1985,4,1482071400,1,116.777634,33.941708,90,27.076721,15.690000,4,0.000000
2777,5,1480925400,1,114.238976,30.611891,58,35.614208,2.750000,0,0.000000
3680,6,1471327860,1,120.765396,31.311472,88,-11.712128,4.620000,0,0.000000
4158,7,1482119760,1,108.364441,30.818899,125,199.296936,0.000000,4,0.000000
5611,8,1478167620,1,121.596626,38.948490,0,19.524536,23.770000,4,2.368560
6309,9,1480491300,1,120.903732,30.710787,297,1.389282,10.280000,0,0.000000
6626,10,1480383180,1,106.607506,29.373436,42,252.591385,1.110000,4,0.000000


In [21]:
test_data

,TERMINALNO,TIME,TRIP_ID,LONGITUDE,LATITUDE,DIRECTION,HEIGHT,SPEED,CALLSTATE
0,301,1476923580,1,122.985168,41.103741,12,39.402588,2.150000,0
294,302,1481711280,1,116.157593,24.658466,174,113.864014,7.770000,4
1022,303,1471742280,1,112.922943,28.820982,315,30.797756,10.760000,0
1985,304,1482071400,1,116.777634,33.941708,90,27.076721,15.690000,4
2777,305,1480925400,1,114.238976,30.611891,58,35.614208,2.750000,0
3680,306,1471327860,1,120.765396,31.311472,88,-11.712128,4.620000,0
4158,307,1482119760,1,108.364441,30.818899,125,199.296936,0.000000,4
5611,308,1478167620,1,121.596626,38.948490,0,19.524536,23.770000,4
6309,309,1480491300,1,120.903732,30.710787,297,1.389282,10.280000,0
6626,310,1480383180,1,106.607506,29.373436,42,252.591385,1.110000,4


In [22]:
data.drop_duplicates(inplace=True,subset='TERMINALNO')

In [23]:
# 切割训练集和测试集
train = data[0:len(train_data)]
test = data[len(train_data):]

In [24]:
train = pd.merge(train,feature,how='left',on='TERMINALNO')
test = pd.merge(test,feature,how='left',on='TERMINALNO')

In [25]:
train.columns

Index(['CALLSTATE', 'DIRECTION', 'HEIGHT', 'LATITUDE', 'LONGITUDE', 'SPEED',
       'TERMINALNO', 'TIME', 'TRIP_ID', 'Y', 'time', 'date', 'hour', 'minute',
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call0', 'call1', 'call_ratio_0', 'call_ratio_1'],
      dtype='object')

In [26]:
test.columns

Index(['CALLSTATE', 'DIRECTION', 'HEIGHT', 'LATITUDE', 'LONGITUDE', 'SPEED',
       'TERMINALNO', 'TIME', 'TRIP_ID', 'Y', 'time', 'date', 'hour', 'minute',
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call0', 'call1', 'call_ratio_0', 'call_ratio_1'],
      dtype='object')

In [27]:
print(train.shape)
print(test.shape)

(100, 32)
(100, 32)


In [28]:
train.head()

,CALLSTATE,DIRECTION,HEIGHT,LATITUDE,LONGITUDE,SPEED,TERMINALNO,TIME,TRIP_ID,Y,...,heg_min,heg_mean,heg,vol,sp_max,sp_mean,call0,call1,call_ratio_0,call_ratio_1
0,0,12,39.402588,41.103741,122.985168,2.15,1,1476923580,1,0.0,...,-7.248444,47.848093,231.318024,1030.594264,32.779999,17.489840,144.0,150.0,0.489796,0.510204
1,4,174,113.864014,24.658466,116.157593,7.77,2,1481711280,1,0.0,...,-112.462982,71.400718,638.763519,18856.694241,36.119999,9.287734,289.0,439.0,0.396978,0.603022
2,0,315,30.797756,28.820982,112.922943,10.76,3,1471742280,1,0.0,...,-10.797699,43.942672,136.545990,887.675299,25.440001,7.987331,838.0,125.0,0.870197,0.129803
3,4,90,27.076721,33.941708,116.777634,15.69,4,1482071400,1,0.0,...,-14.475311,31.659250,130.360809,13.037197,33.310001,6.312753,181.0,611.0,0.228535,0.771465
4,0,58,35.614208,30.611891,114.238976,2.75,5,1480925400,1,0.0,...,-14.855835,29.461915,132.558411,72.227885,53.480000,7.695846,903.0,0.0,1.000000,0.000000


In [29]:
test.head()

,CALLSTATE,DIRECTION,HEIGHT,LATITUDE,LONGITUDE,SPEED,TERMINALNO,TIME,TRIP_ID,Y,...,heg_min,heg_mean,heg,vol,sp_max,sp_mean,call0,call1,call_ratio_0,call_ratio_1
0,0,12,39.402588,41.103741,122.985168,2.15,301,1476923580,1,NaN,...,-7.248444,47.848093,231.318024,1030.594264,32.779999,17.489840,144.0,150.0,0.489796,0.510204
1,4,174,113.864014,24.658466,116.157593,7.77,302,1481711280,1,NaN,...,-112.462982,71.400718,638.763519,18856.694241,36.119999,9.287734,289.0,439.0,0.396978,0.603022
2,0,315,30.797756,28.820982,112.922943,10.76,303,1471742280,1,NaN,...,-10.797699,43.942672,136.545990,887.675299,25.440001,7.987331,838.0,125.0,0.870197,0.129803
3,4,90,27.076721,33.941708,116.777634,15.69,304,1482071400,1,NaN,...,-14.475311,31.659250,130.360809,13.037197,33.310001,6.312753,181.0,611.0,0.228535,0.771465
4,0,58,35.614208,30.611891,114.238976,2.75,305,1480925400,1,NaN,...,-14.855835,29.461915,132.558411,72.227885,53.480000,7.695846,903.0,0.0,1.000000,0.000000


In [30]:
all_feature_list = ['CALLSTATE', 'DIRECTION', 'HEIGHT', 'LATITUDE', 'LONGITUDE', 'SPEED',
       'TERMINALNO', 'TIME', 'TRIP_ID', 'Y', 'time', 'date', 'hour', 'minute',
       'trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call0', 'call1', 'call_ratio_0', 'call_ratio_1']

use_feature_list = ['trip_max', 'lon_max', 'lon_min', 'lon', 'lat_max', 'lat_min', 'lat',
       'heg_max', 'heg_min', 'heg_mean', 'heg', 'vol', 'sp_max', 'sp_mean',
       'call_ratio_0', 'call_ratio_1']

In [31]:
from sklearn.model_selection import train_test_split
train_train,train_val = train_test_split(train,test_size=0.2,random_state = 42)
train_train[use_feature_list].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 55 to 51
Data columns (total 16 columns):
trip_max        80 non-null int64
lon_max         80 non-null float64
lon_min         80 non-null float64
lon             80 non-null float64
lat_max         80 non-null float64
lat_min         80 non-null float64
lat             80 non-null float64
heg_max         80 non-null float64
heg_min         80 non-null float64
heg_mean        80 non-null float64
heg             80 non-null float64
vol             80 non-null float64
sp_max          80 non-null float64
sp_mean         80 non-null float64
call_ratio_0    80 non-null float64
call_ratio_1    80 non-null float64
dtypes: float64(15), int64(1)
memory usage: 10.6 KB


In [32]:
import lightgbm as lgb

In [33]:
lgbmodel = lgb.LGBMRegressor(num_leaves=3, max_depth=-1, n_estimators=2000, n_jobs=20)

In [34]:
lgbmodel.fit(X=train_train[use_feature_list],y=train_train['Y'],eval_set=(train_val[use_feature_list],train_val['Y']),early_stopping_rounds=200)

[1]	valid_0's l2: 0.178145
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's l2: 0.174869
[3]	valid_0's l2: 0.173608
[4]	valid_0's l2: 0.176591
[5]	valid_0's l2: 0.185729
[6]	valid_0's l2: 0.186291
[7]	valid_0's l2: 0.180017
[8]	valid_0's l2: 0.189263
[9]	valid_0's l2: 0.198439
[10]	valid_0's l2: 0.199995
[11]	valid_0's l2: 0.194814
[12]	valid_0's l2: 0.195302
[13]	valid_0's l2: 0.198569
[14]	valid_0's l2: 0.197882
[15]	valid_0's l2: 0.202257
[16]	valid_0's l2: 0.196909
[17]	valid_0's l2: 0.203637
[18]	valid_0's l2: 0.211287
[19]	valid_0's l2: 0.214005
[20]	valid_0's l2: 0.207423
[21]	valid_0's l2: 0.209725
[22]	valid_0's l2: 0.210181
[23]	valid_0's l2: 0.219681
[24]	valid_0's l2: 0.219688
[25]	valid_0's l2: 0.218751
[26]	valid_0's l2: 0.227341
[27]	valid_0's l2: 0.222412
[28]	valid_0's l2: 0.2225
[29]	valid_0's l2: 0.218618
[30]	valid_0's l2: 0.225154
[31]	valid_0's l2: 0.221327
[32]	valid_0's l2: 0.222726
[33]	valid_0's l2: 0.223416
[34]	valid_0's l2: 0.227

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       learning_rate=0.1, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=20, num_leaves=3, objective=None, random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=1)

In [35]:
train_val['pred']=lgbmodel.predict(train_val[use_feature_list])

/home/alanmaxwell/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
train_val[['Y','pred']]

,Y,pred
83,0.000000,0.412516
53,0.000000,0.375810
70,0.000000,0.480185
45,0.000000,0.502061
44,0.000000,0.265574
39,0.000000,0.482828
22,0.000000,0.302281
80,0.000000,0.286975
10,0.000000,0.412516
0,0.000000,0.391826


In [37]:
train_train['pred']=lgbmodel.predict(train_train[use_feature_list])

/home/alanmaxwell/anaconda3/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
train_train[['Y','pred']]

,Y,pred
55,0.000000,0.302281
88,0.000000,0.286975
26,0.000000,0.369950
42,0.000000,0.302281
69,20.742695,0.593064
15,0.000000,0.412516
40,0.000000,0.378453
96,0.845866,0.323681
9,0.000000,0.375810
72,0.000000,0.480185
